In [ ]:
import os

import tensorflow
from itables import init_notebook_mode
from keras.callbacks import TensorBoard
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.metrics import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.data.Dataset as dt

init_notebook_mode(all_interactive=True)

required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

In [ ]:
DATASET_CSV = os.path.abspath(
    "./data/processed/emptyFitdetectionFit/28570Images_0_003_threshold.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

dataset = dt.load_from_csv(DATASET_CSV)

dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataframe = dataset[dataset["subset"] == "train"]
validation_dataframe = dataset[dataset["subset"] == "validation"]
test_dataframe = dataset[dataset["subset"] == "test"]

In [10]:
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)

MODEL_NAME = "VGG16_custom"
VERSION = "v1.0.0"

EPOCHS = 10
BATCH_SIZE = 16
SEED = 42

In [ ]:
train_ImageDataGenerator = ImageDataGenerator(
    rescale=1.0 / 255,
    horizontal_flip=True,
    vertical_flip=True,
)
train_images = train_ImageDataGenerator.flow_from_dataframe(
    dataframe=train_dataframe,
    x_col="file_name",
    y_col="binary_label",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=True,
    seed=SEED,
)

other_ImageDataGenerator = ImageDataGenerator(
    rescale=1.0 / 255,
)
validation_images = other_ImageDataGenerator.flow_from_dataframe(
    dataframe=validation_dataframe,
    x_col="file_name",
    y_col="binary_label",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=True,
    seed=SEED,
)
test_images = other_ImageDataGenerator.flow_from_dataframe(
    dataframe=test_dataframe,
    x_col="file_name",
    y_col="binary_label",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    seed=SEED,
)


In [12]:
input_tensor = Input(shape=IMAGE_SHAPE)
# 1st block
x = Conv2D(64, (3, 3), activation="relu", padding="same", name="conv1a")(input_tensor)
x = Conv2D(64, (3, 3), activation="relu", padding="same", name="conv1b")(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name="pool1")(x)
# 2nd block
x = Conv2D(128, (3, 3), activation="relu", padding="same", name="conv2a")(x)
x = Conv2D(128, (3, 3), activation="relu", padding="same", name="conv2b")(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name="pool2")(x)
# 3rd block
x = Conv2D(256, (3, 3), activation="relu", padding="same", name="conv3a")(x)
x = Conv2D(256, (3, 3), activation="relu", padding="same", name="conv3b")(x)
x = Conv2D(256, (3, 3), activation="relu", padding="same", name="conv3c")(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name="pool3")(x)
# 4th block
x = Conv2D(512, (3, 3), activation="relu", padding="same", name="conv4a")(x)
x = Conv2D(512, (3, 3), activation="relu", padding="same", name="conv4b")(x)
x = Conv2D(512, (3, 3), activation="relu", padding="same", name="conv4c")(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name="pool4")(x)
# 5th block
x = Conv2D(512, (3, 3), activation="relu", padding="same", name="conv5a")(x)
x = Conv2D(512, (3, 3), activation="relu", padding="same", name="conv5b")(x)
x = Conv2D(512, (3, 3), activation="relu", padding="same", name="conv5c")(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name="pool5")(x)
# full connection
x = Flatten()(x)
x = Dense(4096, activation="relu", name="fc6")(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation="relu", name="fc7")(x)
x = Dropout(0.5)(x)
output_tensor = Dense(1, activation="sigmoid", name="fc8")(x)

model = Model(input_tensor, output_tensor)

In [13]:
model.compile(
    # optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.00001),
    optimizer=tensorflow.keras.optimizers.legacy.Adam(learning_rate=0.00001),
    loss=tensorflow.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall(), AUC()],
    callbacks=[TensorBoard(log_dir=f"./logs/{MODEL_NAME}/{VERSION}")],
)

In [ ]:
with tensorflow.device("/GPU:0"):
    history = model.fit(
        train_images,
        steps_per_epoch=train_images.samples // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=validation_images,
        validation_steps=validation_images.samples // BATCH_SIZE,
        verbose=1,
    )